In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import os
import pandas
import matplotlib.pyplot as plt
import pickle
import cv2

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Input, Flatten, Dense, Reshape, Conv2DTranspose
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [4]:
path ="/content/drive/MyDrive/CS338 - Nhận dạng/Seminar/Code/Denoising Autoencoder/dirty_document"

In [5]:
train_img = sorted(os.listdir(os.path.join(path, 'train')))
train_cleaned_img = sorted(os.listdir(os.path.join(path, 'train_cleaned')))
test_img = sorted(os.listdir(os.path.join(path, 'test')))

In [6]:
input_shape = (420, 540, 1)

In [7]:
def process_image(path):
    img = cv2.imread(path)
    img = np.asarray(img, dtype="float32")
    img = cv2.resize(img, (input_shape[1], input_shape[0]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255.0
    img = np.reshape(img, input_shape)
    
    return img

In [8]:
train = []
train_cleaned = []
test = []

directory = os.path.join(path, 'train')  
for f in sorted(os.listdir(directory)):
    train.append(process_image(os.path.join(directory, f)))
                               
directory = os.path.join(path, 'train_cleaned')                               
for f in sorted(os.listdir(directory)):
    train_cleaned.append(process_image(os.path.join(directory, f)))

directory = os.path.join(path, 'test')                               
for f in sorted(os.listdir(directory)):
    test.append(process_image(os.path.join(directory, f)))

In [9]:
plt.figure(figsize=(15,25))
for i in range(0,8,2):
    plt.subplot(4,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train[i][:,:,0], cmap='gray')
    plt.title('Noise image: {}'.format(train_img[i]))
    
    plt.subplot(4,2,i+2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_cleaned[i][:,:,0], cmap='gray')
    plt.title('Denoised image: {}'.format(train_img[i]))

plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [10]:
from sklearn.model_selection import train_test_split

X_train = np.asarray(train)
y_train = np.asarray(train_cleaned)
X_test = np.asarray(test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [11]:
if not os.path.exists('/content/drive/MyDrive/CS338 - Nhận dạng/Seminar/Code/Denoising Autoencoder/dirty_document/dirty_img_data.dat'):
    with open("/content/drive/MyDrive/CS338 - Nhận dạng/Seminar/Code/Denoising Autoencoder/dirty_document/dirty_img_data.dat", "wb") as f:
        pickle.dump([X_train, X_val, y_train, y_val], f)
else:
    with open("/content/drive/MyDrive/CS338 - Nhận dạng/Seminar/Code/Denoising Autoencoder/dirty_document/dirty_img_data.dat", "rb") as f:
        arr = pickle.load(f)
        X_train, X_val, y_train, y_val = arr[0], arr[1], arr[2], arr[3]

In [12]:
batch_size = 32
kernel_size = 3
latent_dim = 16
layer_filters = [
    #{"filters":32, "kernel_size":3},
    {"filters":64, "kernel_size":3}, 
    {"filters":128, "kernel_size":3},
    {"filters":256, "kernel_size":3}
]

In [13]:
def model():
    input_layer = Input(shape=input_shape) 

    # encoding
    x = input_layer
    for layer_f in layer_filters:
        x = Conv2D(layer_f["filters"], (layer_f["kernel_size"], layer_f["kernel_size"]), 
                   activation='relu', padding='same')(x)
    
    x = BatchNormalization()(x)

    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Dropout(0.5)(x)

    # decoding
    for layer_f in layer_filters[::-1]:
        x = Conv2D(layer_f["filters"], (layer_f["kernel_size"], layer_f["kernel_size"]), 
                   activation='relu', padding='same')(x)
        
    x = BatchNormalization()(x)

    x = UpSampling2D((2, 2))(x)

    output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    autoencoder = Model(inputs=[input_layer], outputs=[output_layer])
    autoencoder.compile(optimizer='adam' , loss='mean_squared_error', metrics=['mae'])
    return autoencoder

In [14]:
model = model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 420, 540, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 420, 540, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 420, 540, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 420, 540, 256)     295168    
                                                                 
 batch_normalization (BatchN  (None, 420, 540, 256)    1024      
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 210, 270, 256)    0         
 )                                                           

In [15]:
model_weights_file_path = "/content/drive/MyDrive/CS338 - Nhận dạng/Seminar/Code/Denoising Autoencoder/dirty_document/Best_model_weights.hdf5"

In [16]:
checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min", save_weights_only=True)
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=20)

In [17]:
callbacks=[checkpoint,es]

epochs=500
batch_size=8

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_data = (X_val, y_val), 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=callbacks)

Epoch 1/500


In [ ]:
model.save("/content/drive/MyDrive/nhandang/dirty_image_model.h5")

In [ ]:
epoch_loss = history.history['loss']
epoch_val_loss = history.history['val_loss']
epoch_mae = history.history['mae']
epoch_val_mae = history.history['val_mae']

plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
plt.plot(range(0,len(epoch_loss)), epoch_loss, 'b-', linewidth=2, label='Train Loss')
plt.plot(range(0,len(epoch_val_loss)), epoch_val_loss, 'r-', linewidth=2, label='Val Loss')
plt.title('Evolution of loss on train & validation datasets over epochs')
plt.legend(loc='best')

plt.subplot(1,2,2)
plt.plot(range(0,len(epoch_mae)), epoch_mae, 'b-', linewidth=2, label='Train MAE')
plt.plot(range(0,len(epoch_val_mae)), epoch_val_mae, 'r-', linewidth=2,label='Val MAE')
plt.title('Evolution of MAE on train & validation datasets over epochs')
plt.legend(loc='best')

plt.show()

In [ ]:
model.load_weights(model_weights_file_path)

In [ ]:
type(X_test)

In [ ]:
y_test = model.predict(X_test, batch_size=batch_size)


In [ ]:
plt.figure(figsize=(15,25))
for i in range(0,8,2):
    plt.subplot(4,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_test[i+5][:,:,0], cmap='gray')
    plt.title('Noisy image: {}'.format(test_img[i]))
    
    plt.subplot(4,2,i+2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(y_test[i+5][:,:,0], cmap='gray')
    plt.title('Denoised by autoencoder: {}'.format(test_img[i]))

plt.show()